In [1]:
from googleapiclient.discovery import build
import pandas as pd

# Substitua YOUR_API_KEY pela sua chave de API
api_key = ''
video_id = ''  # Substitua VIDEO_ID pelo ID do vídeo do qual deseja extrair os comentários e respostas
youtube = build('youtube', 'v3', developerKey=api_key)


In [2]:
def get_comment_threads(video_id):
    comments = []
    response = youtube.commentThreads().list(
        part='snippet,replies',
        videoId=video_id,
        maxResults=100,  # API do YouTube permite até 100 resultados por requisição
        textFormat='plainText'
    ).execute()

    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append({
                'author': comment['authorDisplayName'],
                'message': comment['textDisplay'],
                'date': comment['publishedAt'],
                'reply_to': None  # Indica que é um comentário principal
            })
            
            # Verifica se existem respostas para o comentário
            if 'replies' in item and item['replies']['comments']:
                for reply in item['replies']['comments']:
                    reply_snippet = reply['snippet']
                    comments.append({
                        'author': reply_snippet['authorDisplayName'],
                        'message': reply_snippet['textDisplay'],
                        'date': reply_snippet['publishedAt'],
                        'reply_to': comment['authorDisplayName']  # Indica a quem a resposta se destina
                    })

        # Verifica se existe uma próxima página de comentários
        if 'nextPageToken' in response:
            response = youtube.commentThreads().list(
                part='snippet,replies',
                videoId=video_id,
                pageToken=response['nextPageToken'],
                maxResults=100,
                textFormat='plainText'
            ).execute()
        else:
            break

    return comments

# Extrai os comentários e respostas
comments_data = get_comment_threads(video_id)

# Cria um DataFrame com os dados
df = pd.DataFrame(comments_data)

# Exporta o DataFrame para um arquivo CSV
csv_file_name = 'youtube_comments_with_replies.csv'
df.to_csv(csv_file_name, index=False)

print(f'Comentários e respostas exportados para {csv_file_name}.')

Comentários e respostas exportados para youtube_comments_with_replies.csv.
